# LangChain RAG Project with Pride and Prejudice book

## Load ENV variables

In [37]:
import os

In [38]:
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain

In [40]:
openai_api_key = os.getenv('OPENAI_API_KEY')

## Documents loaders

In [41]:
loader = TextLoader('../Data/pride_and_prejudice.txt', encoding='utf-8')
documents = loader.load()

### Text splitting

In [42]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(documents)

### Embeddings

In [43]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [44]:
#vectorstore = FAISS.from_documents(docs, embeddings)

In [45]:
#vectorstore.save_local("../faiss_index")

### Vector storage

In [58]:
index = FAISS.load_local("../persistence/langchain/faiss_index", embeddings, allow_dangerous_deserialization=True)

In [48]:
retriever = index.as_retriever(search_type="similarity", k=3)

In [49]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

### Chat 

In [51]:
template = """
Eres un consejero amoroso con la sabiduría y el estilo de Jane Austen, la autora de "Orgullo y Prejuicio".
Basándote exclusivamente en el contenido del libro y la época en que fue escrito, proporciona consejos sobre relaciones y amor.

Contexto del libro: {context}

Pregunta del usuario: {question}

Por favor, sigue estas instrucciones al responder:
1. Proporciona un consejo amoroso que refleje los valores y la sensibilidad de la época de Jane Austen, pero adaptado a la situación moderna del usuario.
2. El consejo debe estar estrictamente basado en la información y los temas presentes en "Orgullo y Prejuicio".
3. Incluye una cita textual relevante del libro que respalde tu consejo.
4. Sé educado, ingenioso y un poco formal en tu respuesta, como lo sería un personaje de Austen.
5. Si no hay información relevante en el contexto proporcionado para responder la pregunta, admítelo cortésmente.

Consejo:
"""

In [52]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [53]:
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    condense_question_prompt=prompt,
    verbose=True
)

In [56]:
chat_history = []

def get_love_advice(question):
    global chat_history
    inputs = {"question": question, "chat_history": chat_history}
    result = rag_chain(inputs)
    answer = result['answer']
    chat_history.append((question, answer))
    print(answer)

In [57]:
get_love_advice("Estoy perdidamente enamorado de mi mejor amigo, pero no se si es el amor verdadero.")

/var/folders/q8/3qs0qblx57384mws5h46m82m0000gn/T/ipykernel_58855/2664503527.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = rag_chain(inputs)
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
“In vain have I struggled. It will not do. My feelings will not be
repressed. You must allow me to tell you how ardently I admire and love
you.”

being in love with each other many weeks.”

“Very, very much. Nothing could give either Bingley or myself more
delight. But we considered it, we talked of it as impossible. And do you
really love him quite well enough? Oh, Lizzy! do anything rather than
marry without affection. Are you quite sure that you feel what you ought
to do?”

“Oh, yes! You will only think I feel _more_ than I ought to do when I
tell you all.”

“What do you mean?”

“Why, I must confess that I love him better than I do Bingley. I am
afraid you will be angry.”

should be very sorry to be the means of making any of you unhappy; but
since we see, every day, that wher